In [ ]:
# 匯入套件
from bs4 import BeautifulSoup as bs
import requests as req
from pprint import pprint

# 一般用法
---

In [ ]:
# 取得新聞列表
url = "https://www.ptt.cc/bbs/NBA/index.html" 

# 用 requests 的 get 方法把網頁抓下來
res = req.get(url) 

# 指定 lxml 作為解析器
soup = bs(res.text, "lxml") 

# 建立 list 來放置列表資訊
list_posts = []

In [ ]:
# 清空放置列表資訊的變數
list_posts.clear()

# 取得 列表 的文字與超連結
for a in soup.select('div.r-ent div.title a[href]'):
    print(a.get_text())
    print(a['href']) # 或是 a.get('href') 
    
    # 加入列表資訊
    list_posts.append({
        'title': a.get_text(),
        'link': 'https://www.ptt.cc' + a['href']
    })

In [ ]:
# 走訪每一個 a link，整合網頁內文
for index, obj in enumerate(list_posts):
    res_ = req.get(obj['link'])
    soup_ = bs(res_.text, "lxml")
    
    # 去掉 div.article-metaline (作者、標題、時間…等)
    for div in soup_.select('div[class^="article-metaline"]'):
        div.decompose()
        
    # 去掉 div.push (推文: 推、→、噓) (判斷元素是否存在)
    if len( soup_.select('div.push') ) > 0:
        for div in soup_.select('div.push'):
            div.decompose()
    
    # 取得實際需要的內容 (類似 JavaScript 的 innerHTML)
    html = soup_.select_one('div#main-content').decode_contents()
    # html = str(soup_.select_one('div#main-content')) # 類似 JavaScript outerHTML
    
    # 預覽列表和內文
    print(obj['title'])
    print(obj['link'])
    print(html)
    print("=" * 50)
    
    # 整合到列表資訊的變數當中
    list_posts[index]['html'] = html


# 預覽所有結果
pprint(list_posts)

# 思考
- 如何取得**多個分頁**的內容?
  - 觀察分頁數字在網址的呈現方式
  - 將觀察到的分頁數字嵌入對應的網址當中

In [ ]:
# 清空放置列表資訊的變數
list_posts.clear()

# 起始頁數
init_page = 6503

# 最新頁數
latest_page = 6504

# 在已經知道分頁數的情況下
for page in range(init_page, latest_page + 1):
    
    # 取得新聞列表
    url = f"https://www.ptt.cc/bbs/NBA/index{page}.html" 

    # 用 requests 的 get 方法把網頁抓下來
    res = req.get(url) 

    # 指定 lxml 作為解析器
    soup = bs(res.text, "lxml") 
    
    # 取得 列表 的文字與超連結
    for a in soup.select('div.r-ent div.title a[href]'):
        # 加入列表資訊
        list_posts.append({
            'title': a.get_text(),
            'link': 'https://www.ptt.cc' + a['href']
        })
        
# 走訪每一個 a link，整合網頁內文
for index, obj in enumerate(list_posts):
    res_ = req.get(obj['link'])
    soup_ = bs(res_.text, "lxml")

    # 去掉 div.article-metaline (作者、標題、時間…等)
    for div in soup_.select('div[class^="article-metaline"]'):
        div.decompose()
        
    # 去掉 div.push (推文: 推、→、噓) (判斷去掉元素是否存在)
    if len( soup_.select('div.push') ) > 0:
        for div in soup_.select('div.push'):
            div.decompose()

    # 取得實際需要的內容 (類似 JavaScript 的 innerHTML)
    html = soup_.select_one('div#main-content').decode_contents()

    # 整合到列表資訊的變數當中
    list_posts[index]['html'] = html

        
# 預覽所有結果
pprint(list_posts)